### **Load Environment variables from .env file**

In [20]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch
from langchain.document_loaders import TextLoader
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from dotenv import load_dotenv
from IPython.display import display, HTML
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_GPT35_DEPLOYMENT_NAME = os.getenv("OPENAI_GPT35_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME  = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")


GRAPHRAG_API_KEY = os.getenv("GRAPHRAG_API_KEY")
GRAPHRAG_LLM_MODEL = os.getenv("GRAPHRAG_LLM_MODEL")
GRAPHRAG_API_BASE= os.getenv("GRAPHRAG_API_BASE")
GRAPHRAG_API_VERSION = os.getenv("GRAPHRAG_API_VERSION")
GRAPHRAG_EMBEDDING_TYPE = os.getenv("GRAPHRAG_EMBEDDING_TYPE")
GRAPHRAG_EMBEDDING_DEPLOYMENT_NAME = os.getenv("GRAPHRAG_EMBEDDING_DEPLOYMENT_NAME")

def init_llm(model=OPENAI_GPT35_DEPLOYMENT_NAME,
             deployment_name=OPENAI_GPT35_DEPLOYMENT_NAME,
             openai_api_version="2024-02-15-preview",
             temperature=0,
             max_tokens=400
             ):

    llm = AzureChatOpenAI(deployment_name=deployment_name,
                            model=model,
                            openai_api_version=openai_api_version,
                            azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
                            temperature=temperature,
                            max_tokens=max_tokens
                            )
    return llm

llm = init_llm()

embeddings = AzureOpenAIEmbeddings(
    model=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME,
    azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
    openai_api_version="2024-02-15-preview",
    chunk_size = 1
)

graphRagllm = ChatOpenAI(
    api_key=GRAPHRAG_API_KEY,
    model=GRAPHRAG_LLM_MODEL,
    api_base = GRAPHRAG_API_BASE,
    api_version = GRAPHRAG_API_VERSION,
    api_type=OpenaiApiType.AzureOpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

### Load the RAG index from the Faiss vector store to memory

In [21]:
# load the vector store to memory
vectorStore = FAISS.load_local("./faiss/faiss_index", embeddings, allow_dangerous_deserialization= True)
retriever = vectorStore.as_retriever(search_type="similarity", search_kwargs={"k": 10})  # returns 2 most similar vectors/documents
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

In [22]:
r = qa.invoke({"query": "How many people are mentioned in the book?"})
display(HTML(r['result']))

### Load the GraphRAG index

In [23]:
# parquet files generated from indexing pipeline
INPUT_DIR = "./output/20240728-172720/artifacts/"
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"


# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [24]:

entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 140


c:\Users\dschlesinger\code\ongoing\azure-openai-code-samples\.venv\lib\site-packages\graphrag\query\indexer_adapters.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].fillna(-1)
c:\Users\dschlesinger\code\ongoing\azure-openai-code-samples\.venv\lib\site-packages\graphrag\query\indexer_adapters.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].astype(int)


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,139,# Stubb and the Pequod's Maritime Adventures\n...,3,4.5,Stubb and the Pequod's Maritime Adventures,The impact severity rating is moderate to high...,"The community centers around Stubb, a key char...","[{'explanation': 'Stubb, as the second mate on...","{\n ""title"": ""Stubb and the Pequod's Mariti...",991b0fc2-063c-43a1-965f-d1beacc3d969
1,140,# The Pequod's Whale Chase\n\nThis report exam...,3,7.5,The Pequod's Whale Chase,The impact severity rating is relatively high ...,This report examines the community centered ar...,[{'explanation': 'Flask is a central figure in...,"{\n ""title"": ""The Pequod's Whale Chase"",\n ...",dda93d78-a8c6-4147-8533-d25f2cbe13a5
2,141,# Tashtego and The Conflict Aboard The Pequod\...,3,7.5,Tashtego and The Conflict Aboard The Pequod,The impact severity rating is relatively high ...,The community is centered around the character...,[{'explanation': 'Tashtego is depicted as a da...,"{\n ""title"": ""Tashtego and The Conflict Abo...",ed60ddc1-b01d-47b8-8693-28dfbb21f091
3,142,# The Boat and the Whale Hunt\n\nThe community...,3,7.5,The Boat and the Whale Hunt,The impact severity rating is high due to the ...,The community's narrative centers around 'The ...,[{'explanation': ''The Boat' is not only a phy...,"{\n ""title"": ""The Boat and the Whale Hunt"",...",67d866e8-5804-482f-a279-a5e1565e91ee
4,100,# Brazil Banks and Right Whales Interaction\n\...,2,3.0,Brazil Banks and Right Whales Interaction,"The impact severity rating is low to moderate,...",The community is centered around the interacti...,[{'explanation': 'Brazil Banks is described as...,"{\n ""title"": ""Brazil Banks and Right Whales...",847adfb1-3f4a-4093-af5d-ea20c9039d8d


In [25]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

In [26]:

context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [27]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [33]:
result = await search_engine.asearch("How many people are mentioned in the book?")

print(result.response)
# # inspect the data used to build the context for the LLM responses
# result.context_data["reports"]
# # inspect number of LLM calls and tokens
# print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

Exception in _map_response_single_batch
Traceback (most recent call last):
  File "c:\Users\dschlesinger\code\ongoing\azure-openai-code-samples\.venv\lib\site-packages\graphrag\query\structured_search\global_search\search.py", line 182, in _map_response_single_batch
    search_response = await self.llm.agenerate(
  File "c:\Users\dschlesinger\code\ongoing\azure-openai-code-samples\.venv\lib\site-packages\langchain_core\language_models\chat_models.py", line 651, in agenerate
    raise exceptions[0]
  File "c:\Users\dschlesinger\code\ongoing\azure-openai-code-samples\.venv\lib\site-packages\langchain_core\language_models\chat_models.py", line 836, in _agenerate_with_cache
    result = await self._agenerate(
  File "c:\Users\dschlesinger\code\ongoing\azure-openai-code-samples\.venv\lib\site-packages\langchain_openai\chat_models\base.py", line 673, in _agenerate
    payload = self._get_request_payload(messages, stop=stop, **kwargs)
  File "c:\Users\dschlesinger\code\ongoing\azure-openai-co

I am sorry but I am unable to answer this question given the provided data.
